<a href="https://colab.research.google.com/github/vladimir-laz/Sentiment_ANALysis/blob/main/catboost_oversampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/vladimir-laz/Sentiment_ANALysis

In [ ]:
!pip install transformers pytorch_lightning torchmetrics omegaconf

In [ ]:
import sys
sys.path.append('./')

In [ ]:
import os
os.chdir('Sentiment_ANALysis')

In [ ]:
# <your imports>
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pytorch_lightning as pl
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, XLMRobertaXLConfig
from transformers import EarlyStoppingCallback

from preprocessing import Preprocessing
from matplotlib import pyplot as plt

# models
from models import BertClassifier

# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")

# preprocessing

In [ ]:
!pip install nltk

In [ ]:
!pip install pymorphy2

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import  pandas as pd
import pymorphy2
import itertools


# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")


import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')
# pymorphy2 анализатор
morph = pymorphy2.MorphAnalyzer()
# стоп слова из nltk
stops = nltk.corpus.stopwords.words('russian')

In [146]:
word_tokenizer = RegexpTokenizer(r'[a-zа-яёЁА-ЯA-Z]+|[^\w\s]|\d+')
sent_tokenizer = lambda sent: nltk.sent_tokenize(sent, language="russian")

In [147]:
def proccess_text(text):
    text= text.lower()
    sents = sent_tokenizer(text)
    words = list(
        itertools.chain.from_iterable(
            word_tokenizer.tokenize_sents(sents)))
    return [x for x in [morph.normal_forms(word)[0] for word in words ]\
            if x not in stops]

In [ ]:
dataframe = pd.read_csv("data/data_ruSentNE.csv")
dataframe.head(10)

In [149]:
data_text = dataframe['text']
for i in tqdm(range(len(dataframe))):
    # print(data_text.loc[i])
    data_text.iloc[i] = ' '.join(proccess_text(data_text.loc[i]))
    

data_text = dataframe['entity']
for i in tqdm(range(len(dataframe))):
    # print(data_text.loc[i])
    data_text.iloc[i] = ' '.join(proccess_text(data_text.loc[i]))
dataframe.to_csv('data/data_ruSentNE_lemmatized.csv', index=False)

# препроцесинг занимает порядка двух часов (из-за лематизации), поэтому 
# рекомендуется загрузить сразу обработанную версию


  0%|          | 0/9482 [00:00<?, ?it/s]

  0%|          | 0/9482 [00:00<?, ?it/s]

In [ ]:
data = pd.read_csv("data/data_ruSentNE_lemmatized.csv")
data.head(10)

In [151]:
def custom_f1(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    f1_scores = f1_score(y_true=labels, y_pred=pred, average=None)
    final_score = (f1_scores[0] + f1_scores[2]) / 2
    return final_score

In [152]:
conf = OmegaConf.load("config.yaml").general
for i in conf.keys():
    print(f'{i}: {conf[i]}')

dataset_name: ruSentNE_lemmatized
batch_size: 16
device: cpu
dataloader_shuffle: True
pretrained_model: bert-base-uncased
num_classes: 3
num_workers: 1
lr: 1e-06
epochs: 2
eps: 1e-08
optimizer: AdamW
eval_steps: 10
logging_steps: 10


# Oversampling SMOTE

In [223]:
preprocessor = Preprocessing()

result = preprocessor.get_datasets()

train_set = result["train_set"]
val_set = result["val_set"]

mask = [sample['attention_mask'].cpu().detach().numpy() for sample in train_set]
input_ids = [sample['input_ids'].cpu().detach().numpy() for sample in train_set]
label = train_set[:]['label'].cpu()

In [166]:
from imblearn.over_sampling import SMOTE
ada = SMOTE(k_neighbors=1)

In [167]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [172]:
x = mask
x = np.array(x)

In [174]:
y = label
len(y)

7585

In [175]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2,random_state=0, stratify=y)

In [176]:
x_res, y_res = ada.fit_resample(x_train, y_train)

In [ ]:
!pip install catboost

In [247]:
from catboost import CatBoostClassifier
model_1 = CatBoostClassifier(loss_function='MultiClass', task_type="GPU", devices='0:1', n_estimators=1)

In [248]:
model_1.fit(x_res, y_res)

Learning rate set to 0.5
0:	learn: 1.0916333	total: 8.87ms	remaining: 0us


In [249]:
y_pred_1 = model_1.predict(x_train)

In [250]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_1))

              precision    recall  f1-score   support

           0       0.19      0.32      0.24       925
           1       0.77      0.29      0.42      4364
           2       0.14      0.52      0.22       779

    accuracy                           0.32      6068
   macro avg       0.37      0.38      0.29      6068
weighted avg       0.60      0.32      0.37      6068



In [251]:
y_pred_test_1 = model_1.predict(x_test)

In [252]:
print(classification_report(y_pred_test_1, y_test))

              precision    recall  f1-score   support

           0       0.31      0.18      0.23       391
           1       0.26      0.71      0.38       404
           2       0.50      0.13      0.21       722

    accuracy                           0.30      1517
   macro avg       0.36      0.34      0.28      1517
weighted avg       0.39      0.30      0.26      1517



In [112]:
!wget https://raw.githubusercontent.com/dialogue-evaluation/RuSentNE-evaluation/main/final_data.csv

--2023-05-30 22:08:40--  https://raw.githubusercontent.com/dialogue-evaluation/RuSentNE-evaluation/main/final_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 596680 (583K) [text/plain]
Saving to: ‘final_data.csv.1’

final_data.csv.1    100%[===================>] 582.70K  --.-KB/s    in 0.007s  

2023-05-30 22:08:40 (79.1 MB/s) - ‘final_data.csv.1’ saved [596680/596680]



In [253]:
import pandas as pd
data = pd.read_csv('final_data.csv', sep='\t')

In [255]:
import pandas as pd
import yaml
from transformers import (
    BertTokenizerFast,
    RobertaTokenizer,
    GPT2Tokenizer,
    DistilBertTokenizer,
    RobertaTokenizerFast,
)
from src.dataset import TextClassificationDataset, AspectBasedSentimentAnalysisDataset
from torch.utils.data import Dataset, DataLoader

In [256]:
# def test_loader(string):

filaname = "final_data.csv"

with open("config.yaml") as f:
    full_config = yaml.load(f, Loader=yaml.FullLoader)
    config = full_config["preprocessing"]

df = pd.read_csv(f"data/{filaname}", delimiter='\t')
# display(df.head())

if config["tokenizer"] == "BertTokenizerFast":
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
elif config["tokenizer"] == "RobertaTokenizerFast":
    tokenizer = RobertaTokenizerFast.from_pretrained(
        "blinoff/roberta-base-russian-v0", max_len=512
    )
elif config["tokenizer"] == "RobertaTokenizer":
    tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
elif config["tokenizer"] == "GPT2Tokenizer":
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
elif config["tokenizer"] == "DistilBertTokenizer":
    tokenizer = DistilBertTokenizer.from_pretrained(
        "distilbert-base-uncased"
    )
else:
    raise ValueError(f"Tokenizer {config['tokenizer']} not supported")
    
test_dataset = AspectBasedSentimentAnalysisDataset(
    texts=df["sentence"].to_numpy(),
    labels=np.ones(df.shape[0]),
    aspect_terms=df["entity"].to_numpy(),
    tokenizer=tokenizer,
)

test_dl = DataLoader(
    test_dataset,
    batch_size=full_config["general"]["batch_size"],
    shuffle=False,
    num_workers=full_config["general"]["num_workers"],
)

In [ ]:
model = model.to("cuda")

with torch.no_grad():
    outputs = []
    for batch in test_dl:
        inputs = np.array(batch["attention_mask"])
        output = model_1.predict(inputs)
        output = np.array(output)
        print(output)
        outputs.append(output)
outputs = np.vstack(outputs)

In [258]:
data["label"] = outputs
(data["label"]-1).to_csv(f"data/submission.csv", sep="\t", index=False, header=False)